# Heating Relay Pattern Analysis

## Objectives:
1. Analyze relay switching patterns for each heating zone
2. Calculate room-specific duty cycles and energy consumption
3. Identify heating schedules and occupancy patterns
4. Optimize switching frequency to reduce wear and improve efficiency
5. Calculate energy consumption by zone and identify optimization opportunities

## Key Analyses:
- Duty cycle statistics by room and time period
- Switching frequency optimization analysis
- Peak demand analysis and load balancing
- Zone coordination opportunities
- Cost optimization potential with dynamic pricing
- Room temperature vs relay state correlation
- Energy consumption patterns by heating zone

In [ ]:
# Import required libraries
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import asyncio
import warnings
warnings.filterwarnings('ignore')

# Add pems_v2 directory to path for imports
sys.path.append(str(Path('../pems_v2').resolve()))

# Import project modules
from analysis.core.data_extraction import DataExtractor
from analysis.analyzers.pattern_analysis import RelayPatternAnalyzer
from config.settings import PEMSSettings
from config.energy_settings import ROOM_CONFIG

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## 1. Load Heating Relay Data

Load relay state data for all heating zones and corresponding room temperature data

In [ ]:
# Initialize settings and extractors
settings = PEMSSettings()
extractor = DataExtractor(settings)
relay_analyzer = RelayPatternAnalyzer()

# Define analysis period (last 60 days for comprehensive pattern analysis)
end_date = datetime.now()
start_date = end_date - timedelta(days=60)

print(f"Analysis period: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")

In [ ]:
# Extract relay and room data
async def load_heating_data():
    """Load heating relay states and room temperature data."""
    print("Loading heating relay data...")
    relay_data = await extractor.extract_relay_states(start_date, end_date)
    
    print("Loading room temperature data...")
    room_data = await extractor.extract_room_temperatures(start_date, end_date)
    
    print("Loading energy consumption data...")
    consumption_data = await extractor.extract_energy_consumption(start_date, end_date)
    
    print("Loading energy price data...")
    try:
        price_data = await extractor.extract_energy_prices(start_date, end_date)
    except Exception as e:
        print(f"Warning: Could not load price data: {e}")
        price_data = None
    
    return relay_data, room_data, consumption_data, price_data

# Load data
relay_data, room_data, consumption_data, price_data = await load_heating_data()

print(f"\nData loaded:")
print(f"  Relay data: {len(relay_data)} rooms")
print(f"  Room temperature data: {len(room_data)} rooms")
print(f"  Consumption records: {len(consumption_data) if consumption_data is not None else 0}")
print(f"  Price records: {len(price_data) if price_data is not None else 0}")

# Display available rooms
if relay_data:
    print(f"\nRooms with relay data: {list(relay_data.keys())}")
if room_data:
    print(f"Rooms with temperature data: {list(room_data.keys())}")

## 2. Data Quality Assessment

Check data quality and completeness for each heating zone

In [ ]:
# Assess data quality for relay data
def assess_relay_data_quality(relay_data, room_data):
    """Assess quality of relay and temperature data."""
    quality_report = []
    
    all_rooms = set(list(relay_data.keys()) + list(room_data.keys()))
    
    for room in all_rooms:
        relay_df = relay_data.get(room, pd.DataFrame())
        temp_df = room_data.get(room, pd.DataFrame())
        
        report = {
            'room': room,
            'relay_records': len(relay_df),
            'temp_records': len(temp_df),
            'relay_missing_pct': relay_df['value'].isnull().sum() / len(relay_df) * 100 if len(relay_df) > 0 else 100,
            'temp_missing_pct': temp_df['temperature'].isnull().sum() / len(temp_df) * 100 if len(temp_df) > 0 and 'temperature' in temp_df.columns else 100,
            'data_completeness': 'GOOD' if len(relay_df) > 1000 and len(temp_df) > 1000 else 'POOR'
        }
        
        # Calculate duty cycle if relay data available
        if len(relay_df) > 0 and 'value' in relay_df.columns:
            report['duty_cycle_pct'] = relay_df['value'].mean() * 100
            report['switch_count'] = (relay_df['value'].diff() != 0).sum()
        
        quality_report.append(report)
    
    return pd.DataFrame(quality_report)

# Generate quality report
quality_df = assess_relay_data_quality(relay_data, room_data)

print("\nHeating System Data Quality Report:")
print("=" * 80)
display(quality_df.round(2))

In [ ]:
# Visualize data availability
if not quality_df.empty:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Data record counts
    quality_df[['relay_records', 'temp_records']].plot(kind='bar', ax=axes[0,0])
    axes[0,0].set_title('Data Record Counts by Room')
    axes[0,0].set_ylabel('Number of Records')
    axes[0,0].set_xticklabels(quality_df['room'], rotation=45, ha='right')
    axes[0,0].legend()
    
    # Missing data percentages
    quality_df[['relay_missing_pct', 'temp_missing_pct']].plot(kind='bar', ax=axes[0,1])
    axes[0,1].set_title('Missing Data Percentage by Room')
    axes[0,1].set_ylabel('Missing Data (%)')
    axes[0,1].set_xticklabels(quality_df['room'], rotation=45, ha='right')
    axes[0,1].legend()
    
    # Duty cycles
    if 'duty_cycle_pct' in quality_df.columns:
        duty_data = quality_df.dropna(subset=['duty_cycle_pct'])
        if not duty_data.empty:
            duty_data.plot(x='room', y='duty_cycle_pct', kind='bar', ax=axes[1,0], color='orange')
            axes[1,0].set_title('Average Duty Cycle by Room')
            axes[1,0].set_ylabel('Duty Cycle (%)')
            axes[1,0].set_xticklabels(duty_data['room'], rotation=45, ha='right')
    
    # Switch frequency
    if 'switch_count' in quality_df.columns:
        switch_data = quality_df.dropna(subset=['switch_count'])
        if not switch_data.empty:
            switch_data.plot(x='room', y='switch_count', kind='bar', ax=axes[1,1], color='red')
            axes[1,1].set_title('Total Switch Count by Room')
            axes[1,1].set_ylabel('Switch Count')
            axes[1,1].set_xticklabels(switch_data['room'], rotation=45, ha='right')
    
    plt.tight_layout()
    plt.show()

## 3. Daily Heating Patterns

Analyze typical daily heating patterns for each room

In [ ]:
# Analyze daily patterns for rooms with good data
rooms_with_good_data = quality_df[quality_df['data_completeness'] == 'GOOD']['room'].tolist()

print(f"Analyzing daily patterns for {len(rooms_with_good_data)} rooms with good data:")
print(f"Rooms: {rooms_with_good_data}")

# Calculate hourly statistics for each room
daily_patterns = {}

for room in rooms_with_good_data:
    if room in relay_data and not relay_data[room].empty:
        relay_df = relay_data[room].copy()
        
        # Add time features
        relay_df['hour'] = relay_df.index.hour
        relay_df['weekday'] = relay_df.index.weekday
        relay_df['is_weekend'] = relay_df['weekday'].isin([5, 6])
        
        # Calculate hourly patterns
        hourly_stats = {
            'weekday': relay_df[~relay_df['is_weekend']].groupby('hour')['value'].mean(),
            'weekend': relay_df[relay_df['is_weekend']].groupby('hour')['value'].mean(),
            'all_days': relay_df.groupby('hour')['value'].mean()
        }
        
        daily_patterns[room] = hourly_stats

print(f"\nDaily patterns calculated for {len(daily_patterns)} rooms")

In [ ]:
# Plot daily heating patterns
if daily_patterns:
    # Calculate number of subplots needed
    n_rooms = len(daily_patterns)
    cols = 3
    rows = (n_rooms + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows), sharex=True, sharey=True)
    if rows == 1:
        axes = axes.reshape(1, -1)
    
    for idx, (room, patterns) in enumerate(daily_patterns.items()):
        row = idx // cols
        col = idx % cols
        ax = axes[row, col]
        
        # Plot patterns
        if not patterns['weekday'].empty:
            ax.plot(patterns['weekday'].index, patterns['weekday'].values * 100, 
                   label='Weekdays', linewidth=2, marker='o', markersize=4)
        if not patterns['weekend'].empty:
            ax.plot(patterns['weekend'].index, patterns['weekend'].values * 100, 
                   label='Weekends', linewidth=2, marker='s', markersize=4)
        
        ax.set_title(f'Room: {room}')
        ax.set_ylabel('Duty Cycle (%)')
        ax.set_xlabel('Hour of Day')
        ax.set_xlim(0, 23)
        ax.set_ylim(0, 100)
        ax.grid(True, alpha=0.3)
        ax.legend()
    
    # Hide unused subplots
    for idx in range(n_rooms, rows * cols):
        row = idx // cols
        col = idx % cols
        axes[row, col].set_visible(False)
    
    plt.suptitle('Daily Heating Patterns by Room', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    # Print peak heating hours
    print("\nPeak Heating Hours Analysis:")
    print("=" * 60)
    for room, patterns in daily_patterns.items():
        if not patterns['all_days'].empty:
            peak_hour = patterns['all_days'].idxmax()
            peak_duty = patterns['all_days'].max() * 100
            avg_duty = patterns['all_days'].mean() * 100
            print(f"{room:15s}: Peak at {peak_hour:02d}:00 ({peak_duty:.1f}%), Avg: {avg_duty:.1f}%")

## 4. Temperature vs Relay Correlation

Analyze correlation between room temperature and heating relay states

In [ ]:
# Analyze temperature-relay correlation for each room
temp_relay_correlations = {}

for room in rooms_with_good_data:
    if room in relay_data and room in room_data:
        relay_df = relay_data[room]
        temp_df = room_data[room]
        
        if not relay_df.empty and not temp_df.empty and 'temperature' in temp_df.columns:
            # Merge data on timestamp (resample to hourly for cleaner analysis)
            relay_hourly = relay_df['value'].resample('H').mean()
            temp_hourly = temp_df['temperature'].resample('H').mean()
            
            merged = pd.merge(relay_hourly, temp_hourly, left_index=True, right_index=True, how='inner')
            
            if len(merged) > 24:  # Need sufficient data for analysis
                correlation = merged['value'].corr(merged['temperature'])
                
                # Calculate temperature statistics when heating is on vs off
                heating_on = merged[merged['value'] > 0.5]['temperature']
                heating_off = merged[merged['value'] <= 0.5]['temperature']
                
                temp_relay_correlations[room] = {
                    'correlation': correlation,
                    'temp_when_heating': heating_on.mean() if len(heating_on) > 0 else np.nan,
                    'temp_when_not_heating': heating_off.mean() if len(heating_off) > 0 else np.nan,
                    'temp_range': merged['temperature'].max() - merged['temperature'].min(),
                    'avg_duty_cycle': merged['value'].mean() * 100,
                    'merged_data': merged
                }

# Display correlation results
if temp_relay_correlations:
    print("\nTemperature-Relay Correlation Analysis:")
    print("=" * 80)
    print(f"{'Room':15s} {'Correlation':12s} {'Temp(Heat On)':15s} {'Temp(Heat Off)':16s} {'Temp Range':12s} {'Duty Cycle':12s}")
    print("-" * 80)
    
    for room, stats in temp_relay_correlations.items():
        print(f"{room:15s} {stats['correlation']:11.3f} {stats['temp_when_heating']:14.1f}°C {stats['temp_when_not_heating']:15.1f}°C {stats['temp_range']:11.1f}°C {stats['avg_duty_cycle']:11.1f}%")

In [ ]:
# Visualize temperature-relay relationships
if temp_relay_correlations:
    # Select up to 6 rooms for detailed visualization
    rooms_to_plot = list(temp_relay_correlations.keys())[:6]
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    for idx, room in enumerate(rooms_to_plot):
        if idx < len(axes):
            merged_data = temp_relay_correlations[room]['merged_data']
            
            # Create scatter plot with color coding for relay state
            scatter = axes[idx].scatter(merged_data['temperature'], 
                                     merged_data['value'] * 100,
                                     c=merged_data.index.hour, 
                                     cmap='viridis', 
                                     alpha=0.6, 
                                     s=20)
            
            axes[idx].set_xlabel('Temperature (°C)')
            axes[idx].set_ylabel('Duty Cycle (%)')
            axes[idx].set_title(f'{room}\nCorr: {temp_relay_correlations[room]["correlation"]:.3f}')
            axes[idx].grid(True, alpha=0.3)
            
            # Add colorbar for time of day
            cbar = plt.colorbar(scatter, ax=axes[idx])
            cbar.set_label('Hour of Day')
    
    # Hide unused subplots
    for idx in range(len(rooms_to_plot), len(axes)):
        axes[idx].set_visible(False)
    
    plt.suptitle('Temperature vs Heating Duty Cycle (colored by hour)', fontsize=16)
    plt.tight_layout()
    plt.show()

## 5. Switching Frequency Analysis

Analyze heating relay switching patterns and frequency

In [ ]:
# Analyze switching patterns
switching_analysis = {}

for room in rooms_with_good_data:
    if room in relay_data and not relay_data[room].empty:
        relay_df = relay_data[room].copy()
        
        # Detect state changes
        relay_df['state_change'] = (relay_df['value'].diff() != 0) & (relay_df['value'].diff().notna())
        
        # Calculate switching metrics
        total_switches = relay_df['state_change'].sum()
        analysis_days = (relay_df.index.max() - relay_df.index.min()).days + 1
        switches_per_day = total_switches / analysis_days if analysis_days > 0 else 0
        
        # Calculate on/off cycle durations
        relay_df['cycle_id'] = relay_df['state_change'].cumsum()
        
        cycle_stats = []
        for cycle_id in relay_df['cycle_id'].unique():
            cycle_data = relay_df[relay_df['cycle_id'] == cycle_id]
            if len(cycle_data) > 1:
                duration_minutes = (cycle_data.index.max() - cycle_data.index.min()).total_seconds() / 60
                state = cycle_data['value'].iloc[0]  # State during this cycle
                cycle_stats.append({
                    'duration_minutes': duration_minutes,
                    'state': 'on' if state > 0.5 else 'off'
                })
        
        cycle_df = pd.DataFrame(cycle_stats)
        
        # Calculate cycle statistics
        on_cycles = cycle_df[cycle_df['state'] == 'on']['duration_minutes']
        off_cycles = cycle_df[cycle_df['state'] == 'off']['duration_minutes']
        
        switching_analysis[room] = {
            'total_switches': total_switches,
            'switches_per_day': switches_per_day,
            'avg_on_duration_min': on_cycles.mean() if len(on_cycles) > 0 else 0,
            'avg_off_duration_min': off_cycles.mean() if len(off_cycles) > 0 else 0,
            'median_on_duration_min': on_cycles.median() if len(on_cycles) > 0 else 0,
            'median_off_duration_min': off_cycles.median() if len(off_cycles) > 0 else 0,
            'on_cycles': on_cycles,
            'off_cycles': off_cycles
        }

# Display switching analysis results
if switching_analysis:
    print("\nSwitching Frequency Analysis:")
    print("=" * 90)
    print(f"{'Room':15s} {'Switches/Day':13s} {'Avg ON (min)':13s} {'Avg OFF (min)':14s} {'Med ON (min)':13s} {'Med OFF (min)':14s}")
    print("-" * 90)
    
    for room, stats in switching_analysis.items():
        print(f"{room:15s} {stats['switches_per_day']:12.1f} {stats['avg_on_duration_min']:12.1f} {stats['avg_off_duration_min']:13.1f} {stats['median_on_duration_min']:12.1f} {stats['median_off_duration_min']:13.1f}")

In [ ]:
# Visualize switching patterns
if switching_analysis:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Switches per day comparison
    rooms = list(switching_analysis.keys())
    switches_per_day = [switching_analysis[room]['switches_per_day'] for room in rooms]
    
    axes[0,0].bar(rooms, switches_per_day, color='orange')
    axes[0,0].set_title('Switching Frequency by Room')
    axes[0,0].set_ylabel('Switches per Day')
    axes[0,0].set_xticklabels(rooms, rotation=45, ha='right')
    axes[0,0].grid(True, alpha=0.3)
    
    # Average cycle durations
    avg_on = [switching_analysis[room]['avg_on_duration_min'] for room in rooms]
    avg_off = [switching_analysis[room]['avg_off_duration_min'] for room in rooms]
    
    x = np.arange(len(rooms))
    width = 0.35
    
    axes[0,1].bar(x - width/2, avg_on, width, label='ON Cycles', color='red')
    axes[0,1].bar(x + width/2, avg_off, width, label='OFF Cycles', color='blue')
    axes[0,1].set_title('Average Cycle Durations')
    axes[0,1].set_ylabel('Duration (minutes)')
    axes[0,1].set_xticks(x)
    axes[0,1].set_xticklabels(rooms, rotation=45, ha='right')
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)
    
    # Cycle duration distributions for first room
    if rooms:
        first_room = rooms[0]
        on_cycles = switching_analysis[first_room]['on_cycles']
        off_cycles = switching_analysis[first_room]['off_cycles']
        
        if len(on_cycles) > 0:
            axes[1,0].hist(on_cycles, bins=20, alpha=0.7, color='red', label='ON cycles')
            axes[1,0].axvline(on_cycles.mean(), color='red', linestyle='--', label=f'Mean: {on_cycles.mean():.1f} min')
        
        if len(off_cycles) > 0:
            axes[1,0].hist(off_cycles, bins=20, alpha=0.7, color='blue', label='OFF cycles')
            axes[1,0].axvline(off_cycles.mean(), color='blue', linestyle='--', label=f'Mean: {off_cycles.mean():.1f} min')
        
        axes[1,0].set_title(f'Cycle Duration Distribution - {first_room}')
        axes[1,0].set_xlabel('Duration (minutes)')
        axes[1,0].set_ylabel('Frequency')
        axes[1,0].legend()
        axes[1,0].grid(True, alpha=0.3)
    
    # Duty cycle vs switching frequency scatter
    duty_cycles = [temp_relay_correlations.get(room, {}).get('avg_duty_cycle', 0) for room in rooms]
    
    axes[1,1].scatter(duty_cycles, switches_per_day, s=100, alpha=0.7)
    for i, room in enumerate(rooms):
        axes[1,1].annotate(room, (duty_cycles[i], switches_per_day[i]), 
                          xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    axes[1,1].set_xlabel('Average Duty Cycle (%)')
    axes[1,1].set_ylabel('Switches per Day')
    axes[1,1].set_title('Duty Cycle vs Switching Frequency')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 6. Energy Consumption Analysis

Calculate energy consumption by heating zone and identify optimization opportunities

In [ ]:
# Use room power ratings from energy_settings.py
from config.energy_settings import get_room_power

# Calculate energy consumption for each room
energy_consumption = {}

for room in rooms_with_good_data:
    if room in relay_data and not relay_data[room].empty:
        relay_df = relay_data[room].copy()
        
        # Get power rating for this room using the configuration
        power_rating = get_room_power(room) * 1000  # Convert kW to W
        
        # Calculate instantaneous power consumption
        relay_df['power_w'] = relay_df['value'] * power_rating
        
        # Calculate energy consumption (assuming 15-minute intervals)
        interval_hours = 0.25  # 15 minutes = 0.25 hours
        relay_df['energy_kwh'] = relay_df['power_w'] * interval_hours / 1000
        
        # Daily aggregations
        daily_energy = relay_df['energy_kwh'].resample('D').sum()
        daily_avg_power = relay_df['power_w'].resample('D').mean()
        daily_peak_power = relay_df['power_w'].resample('D').max()
        
        energy_consumption[room] = {
            'power_rating_w': power_rating,
            'total_energy_kwh': relay_df['energy_kwh'].sum(),
            'avg_daily_energy_kwh': daily_energy.mean(),
            'max_daily_energy_kwh': daily_energy.max(),
            'avg_power_w': relay_df['power_w'].mean(),
            'daily_energy': daily_energy,
            'daily_avg_power': daily_avg_power,
            'analysis_days': (relay_df.index.max() - relay_df.index.min()).days + 1
        }

# Display energy consumption summary
if energy_consumption:
    print("\nEnergy Consumption Analysis:")
    print("=" * 90)
    print(f"{'Room':15s} {'Power Rating':13s} {'Total (kWh)':12s} {'Avg Daily':12s} {'Max Daily':12s} {'Avg Power':12s}")
    print("-" * 90)
    
    total_consumption = 0
    total_power_rating = 0
    
    for room, stats in energy_consumption.items():
        total_consumption += stats['total_energy_kwh']
        total_power_rating += stats['power_rating_w']
        
        print(f"{room:15s} {stats['power_rating_w']:12.0f}W {stats['total_energy_kwh']:11.1f} {stats['avg_daily_energy_kwh']:11.1f} {stats['max_daily_energy_kwh']:11.1f} {stats['avg_power_w']:11.0f}W")
    
    print("-" * 90)
    print(f"{'TOTAL':15s} {total_power_rating:12.0f}W {total_consumption:11.1f}")
    
    # Calculate analysis period
    analysis_days = max([stats['analysis_days'] for stats in energy_consumption.values()])
    print(f"\nAnalysis period: {analysis_days} days")
    print(f"Total heating energy: {total_consumption:.1f} kWh")
    print(f"Average daily heating: {total_consumption/analysis_days:.1f} kWh/day")
    print(f"Total installed power: {total_power_rating/1000:.1f} kW")

In [ ]:
# Visualize energy consumption patterns
if energy_consumption:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Energy consumption by room (pie chart)
    rooms = list(energy_consumption.keys())
    total_energies = [energy_consumption[room]['total_energy_kwh'] for room in rooms]
    
    axes[0,0].pie(total_energies, labels=rooms, autopct='%1.1f%%', startangle=90)
    axes[0,0].set_title('Total Energy Consumption by Room')
    
    # Daily energy consumption over time (stacked area)
    daily_data = pd.DataFrame()
    for room in rooms:
        daily_data[room] = energy_consumption[room]['daily_energy']
    
    daily_data = daily_data.fillna(0)
    
    # Plot 7-day rolling average for cleaner visualization
    daily_data.rolling(7).mean().plot(kind='area', stacked=True, ax=axes[0,1], alpha=0.7)
    axes[0,1].set_title('Daily Energy Consumption (7-day average)')
    axes[0,1].set_ylabel('Energy (kWh/day)')
    axes[0,1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[0,1].grid(True, alpha=0.3)
    
    # Power rating vs actual consumption
    power_ratings = [energy_consumption[room]['power_rating_w']/1000 for room in rooms]
    avg_powers = [energy_consumption[room]['avg_power_w']/1000 for room in rooms]
    
    x = np.arange(len(rooms))
    width = 0.35
    
    axes[1,0].bar(x - width/2, power_ratings, width, label='Rated Power', alpha=0.8)
    axes[1,0].bar(x + width/2, avg_powers, width, label='Avg Consumption', alpha=0.8)
    axes[1,0].set_title('Power Rating vs Average Consumption')
    axes[1,0].set_ylabel('Power (kW)')
    axes[1,0].set_xticks(x)
    axes[1,0].set_xticklabels(rooms, rotation=45, ha='right')
    axes[1,0].legend()
    axes[1,0].grid(True, alpha=0.3)
    
    # Utilization efficiency (avg power / rated power)
    utilization = [(avg_powers[i] / power_ratings[i] * 100) if power_ratings[i] > 0 else 0 
                   for i in range(len(rooms))]
    
    bars = axes[1,1].bar(rooms, utilization, color='green', alpha=0.7)
    axes[1,1].set_title('Heating System Utilization')
    axes[1,1].set_ylabel('Utilization (%)')
    axes[1,1].set_xticklabels(rooms, rotation=45, ha='right')
    axes[1,1].grid(True, alpha=0.3)
    
    # Add value labels on bars
    for bar, util in zip(bars, utilization):
        height = bar.get_height()
        axes[1,1].text(bar.get_x() + bar.get_width()/2., height + 1,
                      f'{util:.1f}%', ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.show()

## 7. Peak Demand Analysis

Analyze peak heating demand and load balancing opportunities

In [ ]:
# Analyze peak demand patterns
if energy_consumption:
    # Aggregate total heating power across all rooms
    total_power_series = pd.Series(dtype=float)
    
    for room in rooms_with_good_data:
        if room in relay_data and room in energy_consumption:
            relay_df = relay_data[room]
            power_rating = energy_consumption[room]['power_rating_w']
            room_power = relay_df['value'] * power_rating
            
            if total_power_series.empty:
                total_power_series = room_power
            else:
                total_power_series = total_power_series.add(room_power, fill_value=0)
    
    if not total_power_series.empty:
        # Resample to hourly for analysis
        hourly_power = total_power_series.resample('H').mean()
        hourly_peak = total_power_series.resample('H').max()
        
        # Calculate peak demand statistics
        max_simultaneous_power = total_power_series.max()
        avg_power = total_power_series.mean()
        peak_utilization = max_simultaneous_power / total_power_rating * 100
        
        # Find peak demand times
        peak_times = total_power_series[total_power_series > total_power_series.quantile(0.95)]
        
        print("\nPeak Demand Analysis:")
        print("=" * 60)
        print(f"Total installed power: {total_power_rating/1000:.1f} kW")
        print(f"Maximum simultaneous demand: {max_simultaneous_power/1000:.1f} kW")
        print(f"Average power consumption: {avg_power/1000:.1f} kW")
        print(f"Peak utilization: {peak_utilization:.1f}%")
        print(f"Times above 95th percentile: {len(peak_times)} periods")
        
        # Analyze peak times by hour of day
        if len(peak_times) > 0:
            peak_hours = peak_times.index.hour.value_counts().sort_index()
            print(f"\nPeak demand hours:")
            for hour, count in peak_hours.head(5).items():
                print(f"  {hour:02d}:00 - {count} times ({count/len(peak_times)*100:.1f}%)")

In [ ]:
# Visualize peak demand patterns
if 'total_power_series' in locals() and not total_power_series.empty:
    fig, axes = plt.subplots(3, 1, figsize=(15, 12), sharex=True)
    
    # Total power consumption over time
    hourly_power.plot(ax=axes[0], color='blue', alpha=0.7)
    axes[0].axhline(y=max_simultaneous_power, color='red', linestyle='--', 
                   label=f'Max: {max_simultaneous_power/1000:.1f} kW')
    axes[0].axhline(y=avg_power, color='green', linestyle='--', 
                   label=f'Avg: {avg_power/1000:.1f} kW')
    axes[0].set_ylabel('Power (W)')
    axes[0].set_title('Total Heating Power Consumption')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Daily peak power
    daily_peak = total_power_series.resample('D').max()
    daily_peak.plot(ax=axes[1], color='red', marker='o', markersize=4)
    axes[1].set_ylabel('Peak Power (W)')
    axes[1].set_title('Daily Peak Heating Demand')
    axes[1].grid(True, alpha=0.3)
    
    # Hourly peak demand distribution
    hourly_peaks_by_hour = total_power_series.groupby(total_power_series.index.hour).max()
    hourly_peaks_by_hour.plot(kind='bar', ax=axes[2], color='orange', alpha=0.8)
    axes[2].set_ylabel('Peak Power (W)')
    axes[2].set_xlabel('Hour of Day')
    axes[2].set_title('Peak Heating Demand by Hour')
    axes[2].set_xticklabels(range(24), rotation=0)
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Load duration curve
    plt.figure(figsize=(10, 6))
    sorted_power = total_power_series.sort_values(ascending=False)
    hours = np.arange(len(sorted_power)) / len(sorted_power) * 100
    
    plt.plot(hours, sorted_power/1000, linewidth=2)
    plt.axhline(y=max_simultaneous_power/1000, color='red', linestyle='--', alpha=0.7, 
               label=f'Peak: {max_simultaneous_power/1000:.1f} kW')
    plt.axhline(y=avg_power/1000, color='green', linestyle='--', alpha=0.7, 
               label=f'Average: {avg_power/1000:.1f} kW')
    
    plt.xlabel('Time (%)')
    plt.ylabel('Power (kW)')
    plt.title('Heating Load Duration Curve')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

## 8. Cost Optimization Analysis

Analyze heating costs and optimization opportunities with dynamic pricing

In [ ]:
# Cost analysis with energy prices
if price_data is not None and 'total_power_series' in locals():
    # Merge power consumption with price data
    hourly_power_df = hourly_power.to_frame('power_w')
    hourly_prices = price_data.resample('H').mean()
    
    cost_analysis = pd.merge(hourly_power_df, hourly_prices, 
                            left_index=True, right_index=True, how='inner')
    
    if not cost_analysis.empty and 'price_czk_kwh' in cost_analysis.columns:
        # Calculate hourly heating costs
        cost_analysis['energy_kwh'] = cost_analysis['power_w'] / 1000  # Convert to kWh
        cost_analysis['cost_czk'] = cost_analysis['energy_kwh'] * cost_analysis['price_czk_kwh']
        
        # Daily cost aggregation
        daily_costs = cost_analysis['cost_czk'].resample('D').sum()
        daily_energy = cost_analysis['energy_kwh'].resample('D').sum()
        
        # Cost statistics
        total_cost = cost_analysis['cost_czk'].sum()
        avg_daily_cost = daily_costs.mean()
        max_daily_cost = daily_costs.max()
        avg_price = cost_analysis['price_czk_kwh'].mean()
        
        print("\nHeating Cost Analysis:")
        print("=" * 60)
        print(f"Analysis period: {len(daily_costs)} days")
        print(f"Total heating cost: {total_cost:.0f} CZK")
        print(f"Average daily cost: {avg_daily_cost:.1f} CZK")
        print(f"Maximum daily cost: {max_daily_cost:.1f} CZK")
        print(f"Average energy price: {avg_price:.2f} CZK/kWh")
        print(f"Total heating energy: {cost_analysis['energy_kwh'].sum():.1f} kWh")
        print(f"Average cost per kWh: {total_cost/cost_analysis['energy_kwh'].sum():.2f} CZK/kWh")
        
        # Price band analysis
        price_bands = {
            'Low': cost_analysis['price_czk_kwh'] <= cost_analysis['price_czk_kwh'].quantile(0.33),
            'Medium': (cost_analysis['price_czk_kwh'] > cost_analysis['price_czk_kwh'].quantile(0.33)) & 
                     (cost_analysis['price_czk_kwh'] <= cost_analysis['price_czk_kwh'].quantile(0.67)),
            'High': cost_analysis['price_czk_kwh'] > cost_analysis['price_czk_kwh'].quantile(0.67)
        }
        
        print("\nHeating by Price Band:")
        print("-" * 40)
        for band, mask in price_bands.items():
            band_energy = cost_analysis[mask]['energy_kwh'].sum()
            band_cost = cost_analysis[mask]['cost_czk'].sum()
            band_pct = band_energy / cost_analysis['energy_kwh'].sum() * 100
            avg_band_price = cost_analysis[mask]['price_czk_kwh'].mean()
            
            print(f"{band:8s}: {band_energy:6.1f} kWh ({band_pct:4.1f}%) - {band_cost:6.0f} CZK - {avg_band_price:.2f} CZK/kWh")
    
    else:
        print("\nPrice data available but no price_czk_kwh column found")
else:
    print("\nNo price data available for cost analysis")

In [ ]:
# Visualize cost patterns
if 'cost_analysis' in locals() and not cost_analysis.empty:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Daily costs over time
    daily_costs.plot(ax=axes[0,0], color='red', linewidth=2, alpha=0.8)
    daily_costs.rolling(7).mean().plot(ax=axes[0,0], color='darkred', linewidth=2, 
                                       label='7-day average')
    axes[0,0].set_title('Daily Heating Costs')
    axes[0,0].set_ylabel('Cost (CZK/day)')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # Power vs price scatter
    sample_data = cost_analysis.sample(min(1000, len(cost_analysis)))
    scatter = axes[0,1].scatter(sample_data['price_czk_kwh'], sample_data['power_w']/1000, 
                               c=sample_data.index.hour, cmap='viridis', alpha=0.6)
    axes[0,1].set_xlabel('Price (CZK/kWh)')
    axes[0,1].set_ylabel('Power (kW)')
    axes[0,1].set_title('Heating Power vs Energy Price')
    axes[0,1].grid(True, alpha=0.3)
    cbar = plt.colorbar(scatter, ax=axes[0,1])
    cbar.set_label('Hour of Day')
    
    # Hourly cost distribution
    hourly_costs = cost_analysis.groupby(cost_analysis.index.hour)['cost_czk'].sum()
    hourly_costs.plot(kind='bar', ax=axes[1,0], color='orange', alpha=0.8)
    axes[1,0].set_title('Total Heating Cost by Hour')
    axes[1,0].set_xlabel('Hour of Day')
    axes[1,0].set_ylabel('Total Cost (CZK)')
    axes[1,0].set_xticklabels(range(24), rotation=0)
    axes[1,0].grid(True, alpha=0.3)
    
    # Cost vs energy efficiency
    cost_analysis['cost_per_kwh'] = cost_analysis['cost_czk'] / (cost_analysis['energy_kwh'] + 0.001)
    weekly_efficiency = cost_analysis['cost_per_kwh'].resample('W').mean()
    weekly_efficiency.plot(ax=axes[1,1], color='purple', linewidth=2, marker='o')
    axes[1,1].set_title('Weekly Average Cost Efficiency')
    axes[1,1].set_ylabel('Cost per kWh (CZK/kWh)')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 9. Optimization Recommendations

Generate actionable recommendations for heating system optimization

In [ ]:
print("\nHeating Pattern Analysis - Key Insights and Recommendations:")
print("=" * 80)

# Generate insights based on analysis
insights = []
recommendations = []

# Room utilization insights
if energy_consumption:
    high_util_rooms = []
    low_util_rooms = []
    
    for room in energy_consumption.keys():
        power_rating = energy_consumption[room]['power_rating_w']
        avg_power = energy_consumption[room]['avg_power_w']
        utilization = (avg_power / power_rating * 100) if power_rating > 0 else 0
        
        if utilization > 30:
            high_util_rooms.append((room, utilization))
        elif utilization < 10:
            low_util_rooms.append((room, utilization))
    
    if high_util_rooms:
        insights.append(f"High utilization rooms: {', '.join([f'{r}({u:.1f}%)' for r, u in high_util_rooms])}")
        recommendations.append("Consider thermal improvements for high-utilization rooms")
    
    if low_util_rooms:
        insights.append(f"Low utilization rooms: {', '.join([f'{r}({u:.1f}%)' for r, u in low_util_rooms])}")
        recommendations.append("Review heating schedules for low-utilization rooms")

# Switching frequency insights
if switching_analysis:
    high_switching = [(room, stats['switches_per_day']) for room, stats in switching_analysis.items() 
                     if stats['switches_per_day'] > 20]
    
    if high_switching:
        insights.append(f"High switching frequency rooms: {', '.join([f'{r}({s:.1f}/day)' for r, s in high_switching])}")
        recommendations.append("Implement hysteresis control to reduce relay wear")

# Peak demand insights
if 'peak_utilization' in locals():
    insights.append(f"Peak system utilization: {peak_utilization:.1f}%")
    if peak_utilization > 80:
        recommendations.append("Consider load balancing to reduce peak demand")
    elif peak_utilization < 50:
        recommendations.append("System has capacity for additional heating zones")

# Cost optimization insights
if 'cost_analysis' in locals() and not cost_analysis.empty:
    high_price_heating = cost_analysis[cost_analysis['price_czk_kwh'] > cost_analysis['price_czk_kwh'].quantile(0.8)]
    high_price_pct = len(high_price_heating) / len(cost_analysis) * 100
    
    insights.append(f"Heating during high prices: {high_price_pct:.1f}% of time")
    if high_price_pct > 25:
        recommendations.append("Implement price-based heating schedule optimization")
        recommendations.append("Pre-heat rooms during low-price periods")

# Temperature control insights
if temp_relay_correlations:
    poor_correlation_rooms = [room for room, stats in temp_relay_correlations.items() 
                             if abs(stats['correlation']) < 0.3]
    
    if poor_correlation_rooms:
        insights.append(f"Poor temperature control: {', '.join(poor_correlation_rooms)}")
        recommendations.append("Review thermostat calibration for poorly controlled rooms")

# Display insights and recommendations
print("\nKey Insights:")
for i, insight in enumerate(insights, 1):
    print(f"{i}. {insight}")

print("\nOptimization Recommendations:")
for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec}")

# Summary statistics
print("\nSummary Statistics:")
print("-" * 40)
if energy_consumption:
    total_energy = sum([stats['total_energy_kwh'] for stats in energy_consumption.values()])
    analysis_days = max([stats['analysis_days'] for stats in energy_consumption.values()])
    print(f"Total heating energy: {total_energy:.1f} kWh over {analysis_days} days")
    print(f"Average daily consumption: {total_energy/analysis_days:.1f} kWh/day")

if 'total_cost' in locals():
    print(f"Total heating cost: {total_cost:.0f} CZK")
    print(f"Average daily cost: {total_cost/(len(daily_costs) if 'daily_costs' in locals() else 1):.1f} CZK/day")

print(f"\nAnalysis completed for {len(rooms_with_good_data)} heating zones.")

In [ ]:
# Save analysis results for future use
import pickle
from pathlib import Path

# Create results dictionary
heating_analysis_results = {
    'analysis_period': {'start': start_date, 'end': end_date},
    'rooms_analyzed': rooms_with_good_data,
    'daily_patterns': daily_patterns,
    'switching_analysis': switching_analysis,
    'energy_consumption': energy_consumption,
    'temp_relay_correlations': temp_relay_correlations,
    'insights': insights,
    'recommendations': recommendations
}

# Add cost analysis if available
if 'cost_analysis' in locals():
    heating_analysis_results['cost_analysis_summary'] = {
        'total_cost_czk': total_cost,
        'avg_daily_cost_czk': avg_daily_cost,
        'total_energy_kwh': cost_analysis['energy_kwh'].sum(),
        'avg_price_czk_kwh': avg_price
    }

# Save to files
results_dir = Path('./data/processed')
results_dir.mkdir(parents=True, exist_ok=True)

# Save as pickle for programmatic use
with open(results_dir / 'heating_pattern_analysis.pkl', 'wb') as f:
    pickle.dump(heating_analysis_results, f)

# Save summary as text
with open(results_dir / 'heating_analysis_summary.txt', 'w') as f:
    f.write("Heating Pattern Analysis Summary\n")
    f.write("=" * 40 + "\n\n")
    f.write(f"Analysis Period: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}\n")
    f.write(f"Rooms Analyzed: {len(rooms_with_good_data)}\n\n")
    
    f.write("Key Insights:\n")
    for i, insight in enumerate(insights, 1):
        f.write(f"{i}. {insight}\n")
    
    f.write("\nRecommendations:\n")
    for i, rec in enumerate(recommendations, 1):
        f.write(f"{i}. {rec}\n")

print("\nAnalysis results saved to:")
print(f"  - {results_dir / 'heating_pattern_analysis.pkl'}")
print(f"  - {results_dir / 'heating_analysis_summary.txt'}")